In [1]:
from bs4 import BeautifulSoup
from requests import get
import requests
import csv 

In [6]:
def loginandgetcsv(user,password,user1,user2,user3):
    #first create a session 's'
    s = requests.session()
    #this is the data that is sent to login to icheckmovies
    login_data = {'login[username]': user,'login[password]': password,'submit': 'login',}
    #here we send the login data, using the session created before
    s.post("https://www.icheckmovies.com/login/", data=login_data) 
    #once again, whithin the session we get the csv link and call it 'r'
    usernamelist=[user1,user2,user3]
    for item in usernamelist:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file = open(filename, "w")
        file.write(checklist_encode)
        file.close() 

now 'loginandgetcsv' will save a .csv in the parent directory called *username*_checklist.csv

In [7]:
loginandgetcsv(user,password,user1,user2,user3)